# Extract Tesla Stock Data By Web Scraping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

In [3]:
url = "https://finance.yahoo.com/quote/TSLA/history/?period1=1277818200&period2=1758654962"
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
}
time.sleep(2)
session = requests.Session()
response = session.get(url, headers = headers, timeout = 30)
response.status_code

200

In [4]:
with open("tesla_stock.html", "w", encoding = 'utf-8') as f:
    f.write(response.text)

In [5]:
with open("tesla_stock.html", "r", encoding = 'utf-8') as f:
    html_data = f.read()

In [6]:
soup = BeautifulSoup(html_data, 'html.parser')

In [7]:
print(soup.title.string)

Tesla, Inc. (TSLA) Stock Historical Prices & Data - Yahoo Finance


In [8]:
for col in soup.find_all('thead'):
    print(col.text)

Date  Open  High  Low  Close    Close price adjusted for splits.  Adj Close    Adjusted close price adjusted for splits and dividend and/or capital gain distributions.  Volume  


In [9]:
Tesla_Stock_Data = pd.DataFrame(columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume','Dividends','Stock Splits'])
for row in soup.find_all('tbody')[0].find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 7:
       date = cols[0].text
       openn = cols[1].text
       high = cols[2].text
       low = cols[3].text
       close = cols[4].text
       adj_close = cols[5].text
       volume = cols[6].text
       Tesla_Stock_Data = pd.concat([Tesla_Stock_Data, pd.DataFrame({'Date':[date],'Open':[openn],'High':[high],
       'Low':[low],'Close':[close],'Adj Close':[adj_close],'Volume':[volume],'Dividends':0.0, 'Stock Splits':0.0})], ignore_index = True)
Tesla_Stock_Data.loc[Tesla_Stock_Data['Date'] == 'Aug 25, 2022', 'Stock Splits'] = 3.0
Tesla_Stock_Data.loc[Tesla_Stock_Data['Date'] == 'Aug 31, 2020', 'Stock Splits'] = 5.0

In [10]:
Tesla_Stock_Data

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,"Sep 23, 2025",439.88,440.97,423.72,425.85,425.85,"83,422,700",0.0,0.0
1,"Sep 22, 2025",431.11,444.98,429.13,434.21,434.21,"97,108,800",0.0,0.0
2,"Sep 19, 2025",421.82,429.47,421.72,426.07,426.07,"93,131,000",0.0,0.0
3,"Sep 18, 2025",428.87,432.22,416.56,416.85,416.85,"90,454,500",0.0,0.0
4,"Sep 17, 2025",415.75,428.31,409.67,425.86,425.86,"106,133,500",0.0,0.0
...,...,...,...,...,...,...,...,...,...
3828,"Jul 6, 2010",1.33,1.33,1.06,1.07,1.07,"103,003,500",0.0,0.0
3829,"Jul 2, 2010",1.53,1.54,1.25,1.28,1.28,"77,097,000",0.0,0.0
3830,"Jul 1, 2010",1.67,1.73,1.35,1.46,1.46,"123,282,000",0.0,0.0
3831,"Jun 30, 2010",1.72,2.03,1.55,1.59,1.59,"257,806,500",0.0,0.0


In [11]:
Tesla_Stock_Data.to_csv("Tesla_Stock_Data.csv", index = False)

In [12]:
tesladata = pd.read_csv("Tesla_Stock_Data.csv")
tesladata

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,"Sep 23, 2025",439.88,440.97,423.72,425.85,425.85,"83,422,700",0.0,0.0
1,"Sep 22, 2025",431.11,444.98,429.13,434.21,434.21,"97,108,800",0.0,0.0
2,"Sep 19, 2025",421.82,429.47,421.72,426.07,426.07,"93,131,000",0.0,0.0
3,"Sep 18, 2025",428.87,432.22,416.56,416.85,416.85,"90,454,500",0.0,0.0
4,"Sep 17, 2025",415.75,428.31,409.67,425.86,425.86,"106,133,500",0.0,0.0
...,...,...,...,...,...,...,...,...,...
3828,"Jul 6, 2010",1.33,1.33,1.06,1.07,1.07,"103,003,500",0.0,0.0
3829,"Jul 2, 2010",1.53,1.54,1.25,1.28,1.28,"77,097,000",0.0,0.0
3830,"Jul 1, 2010",1.67,1.73,1.35,1.46,1.46,"123,282,000",0.0,0.0
3831,"Jun 30, 2010",1.72,2.03,1.55,1.59,1.59,"257,806,500",0.0,0.0


# Extract Tesla Stock Data Using yfinance API

In [13]:
import yfinance as yf

In [14]:
tesla_stock = yf.Ticker("TSLA")

In [15]:
tesla_data = tesla_stock.history("max")
tesla_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [16]:
tesla_data.reset_index(inplace = True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [17]:
tesla_data['Date'] = tesla_data['Date'].dt.date
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [18]:
tesla_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0
...,...,...,...,...,...,...,...,...
3829,2025-09-18,428.869995,432.220001,416.559998,416.850006,90454500,0.0,0.0
3830,2025-09-19,421.820007,429.470001,421.720001,426.070007,93131000,0.0,0.0
3831,2025-09-22,431.109985,444.980011,429.130005,434.209991,97108800,0.0,0.0
3832,2025-09-23,439.880005,440.970001,423.720001,425.850006,83422700,0.0,0.0
